In [1]:
import os
import datetime
import pandas as pd
import numpy as np
import dask.dataframe as dd
from dask import delayed, compute
from sklearn.cross_validation import train_test_split
import lightgbm as lgb
import gc
import time
import pickle
from tqdm import tqdm,tqdm_pandas
path = "F:\\Pratik\\LearningZone\\Competitions\\Avito_Demand_Prediction_Challenge"
os.chdir(path)

C:\Program Files\Anaconda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = pd.read_csv('train.csv')

In [ ]:
train.info()

In [ ]:
train.head()

In [ ]:
train.tail()

In [16]:
from textblob import TextBlob

In [ ]:
TextBlob('Волгоградская область').translate(to='en')

In [27]:
@delayed
def desc_missing(x):
    '''
    Takes data frame as input, then searches and fills missing description with недостающий
    '''
    if x['description'].isnull().sum()>0:
        print("Description column has missing values, filling up with недостающий")
        x['description'].fillna("недостающий",inplace=True)
        return x
    else:
        return x

#delayed
def translate(x):
    try:
        return TextBlob(x).translate(to="en")
    except:
        return x

#@delayed    
def map_translate(x):
    print("Begining to translate")
    tqdm.pandas(tqdm())
    x['en_desc']=x['description'].map(translate)
    #x['en_desc'] = x['en_desc'].compute()
    print("Done translating decription")
    print("Begining to translate Title")
    x['en_title']=x['title'].map(translate)
    #x['en_title'] = x['en_title'].compute()
    print("Done translating")
    return x

In [ ]:
%time train['en_title'] = train.title.map(lambda x: translate(x)).compute()

In [ ]:
#%time train['en_title'] = (train.apply(lambda x: translate(x['title']), axis=1)).compute() 
%time values = (train.apply(lambda x: translate(list(x['title'])), axis=1)).compute() 

In [ ]:
#values = train['title'].apply(translate).compute()
values = [delayed(translate)(x) for x in train['title']]

In [ ]:
import dask.multiprocessing
%time results = compute(*values, get=dask.multiprocessing.get)

In [ ]:
%time final_val = [x.compute() for x in values]

In [ ]:
[x.compute() for x in values]

In [ ]:

%time for i in values:
    trans_values.append(i.compute())

In [31]:
%time data_comp = delayed(map_translate)(data_train)

Wall time: 1.09 ms


In [33]:
%time data_comp.head().compute()

Begining to translate



0it [00:00, ?it/s]


KeyboardInterrupt: 

In [20]:
data_train =desc_missing(train)
%time data_train = data_train.compute()
# data_comp = map_translate(data_train)
# %time data_comp = data_comp.compute()

Wall time: 247 ms
Begining to translate



0it [00:00, ?it/s]


Done translating decription
Begining to translate Title
Done translating
Wall time: 1min 34s


In [26]:
data_comp['en_title'].head().compute()

AttributeError: 'Series' object has no attribute 'compute'

In [15]:
TextBlob(data_comp.loc[0,'title']).translate(to="en")

NameError: name 'TextBlob' is not defined

In [ ]:
%time data_train['en_desc']= data_train['description'].progress_map(translate)

In [ ]:
%time data_train = data_train.compute()

In [29]:
data_train.drop(['en_desc','en_title'], axis=1, inplace=True)

In [ ]:
#data_comp = pd.DataFrame(columns=['en_desc'])
data_comp = data_train['description'].map(translate)

In [ ]:
%time data_comp = data_comp.compute()